In [ ]:
# Transforming AL file with new directions and news columns (route_id and trip_id)

In [1]:
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# spark = sparkSession
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1608251019542_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
from pyspark.sql.types import *

from pyspark.sql.functions import *

AL_schema = StructType([
    StructField("line_number", StringType()),
    StructField("complement", StringType()),
    StructField("line_id", IntegerType()),
    StructField("direction", IntegerType())
])

for day in range(1,32):
    # Reading AL file
    AL = spark.read.csv("s3://mobility-traces-sp/aux-files/route-files/AL_1510"+str(day)+".txt",header=False,schema=AL_schema)

    trips = spark.read.csv("s3://mobility-traces-sp/aux-files/gtfs/trips.csv",header=True)

    trips = trips.drop("route_id","service_id")

    # Converting direction 1 to 0, and 2 to 1
    AL_new_direction = AL.withColumn("new_direction",when(AL["direction"] == "1", 0).otherwise(1))

    # dropping new_direction column
    AL_direction = AL_new_direction.drop("direction").withColumnRenamed("new_direction","direction")

    # Defining route_id and trip_id based in line_number, complement and direction
    AL_transformed = AL_direction.withColumn('route_id',concat_ws("-",col("line_number"),col("complement"))) \
        .withColumn('trip_id',concat_ws("-",col("route_id"),("direction")))

    # Defining the line_id shape by joining trips.csv and AL file. 
    # The trips not found in trips.csv are eliminated by inner join
    AL_with_shape = AL_transformed.join(trips,on=["trip_id"],how="inner").drop("line_number","complement")
    
    AL_with_shape.write.parquet("s3://mobility-traces-sp/aux-files/route-files-enriched/AL_1510"+str(day)+"/")

    print(AL.count(),AL_with_shape.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2782 2252
2782 2252
2782 2252
2782 2252
2782 2252
2782 2252
2782 2252
2782 2252
2782 2252
2782 2252
2782 2252
2782 2252
2782 2252
2782 2252
2784 2252
2784 2252
2784 2252
2784 2252
2784 2252
2784 2252
2784 2252
2784 2252
2784 2252
2784 2252
2784 2252
2784 2252
2786 2253
2786 2253
2784 2252
2784 2252
2786 2252